In [66]:
library(childesr)
library(purrr)
library(tidyverse)
library(here)

In [67]:
language_codes <- read_csv(here("spacy/language-codes-full_csv.csv"))
spacy_models <- read_csv(here("spacy/SpaCy Models list.csv"))

language_counts = childesr::get_sql_query("SELECT language, count(*) as utterance_count from utterance group by language")


── Column specification ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  `639-3` = col_character(),
  English = col_character()
)



── Column specification ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  language = col_character(),
  `639-3` = col_character(),
  `639-1` = col_character(),
  models = col_character(),
  `model type` = col_character(),
  size_mb = col_double(),
  pipeline = col_character()
)


Using current database version: '2020.1'.



In [83]:
model_counts <- language_counts %>% 
mutate(language = pluck(strsplit(language, " ")),
      all_codes = language) 

#tidy way to do this?
model_counts$language <- unlist(lapply(model_counts$language, "[[", 1))

model_grouped_counts <- model_counts %>% 
group_by(language, all_codes) %>%
summarize(utterance_count = sum(utterance_count)) %>%
ungroup() %>%
arrange(desc(utterance_count, - language)) %>%
left_join(language_codes, by = c("language" = "639-3")) %>%
left_join(spacy_models, by = c("language" = "639-3"))

model_grouped_counts %>% arrange(desc(utterance_count))

`summarise()` has grouped output by 'language'. You can override using the `.groups` argument.



language,all_codes,utterance_count,English,language.y,639-1,models,model type,size_mb,pipeline
<chr>,<list>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
eng,eng,6876924,English,English,en,en_core_web_lg,web,742,"tok2vec, tagger, parser, ner, attribute_ruler, lemmatizer"
deu,deu,1587335,German,German,de,de_core_news_lg,news,546,"tok2vec, tagger, morphologizer, parser, ner, attribute_ruler, lemmatizer"
ind,"ind, jav, und, xmm",911689,Indonesian,NA,NA,NA,NA,NA,NA
fra,fra,871732,French,French,fr,fr_core_news_lg,news,546,"tok2vec, morphologizer, parser, ner, attribute_ruler, lemmatizer"
jpn,jpn,842833,Japanese,Japanese,ja,ja_core_news_lg,news,531,"tok2vec, parser, ner, attribute_ruler"
nld,nld,750538,Dutch; Flemish,Dutch,nl,nl_core_news_lg,news,546,"tok2vec, morphologizer, tagger, parser, ner, attribute_ruler, lemmatizer"
spa,spa,616197,Spanish; Castilian,Spanish,es,es_core_news_lg,news,543,"tok2vec, morphologizer, parser, senter, ner, attribute_ruler, lemmatizer"
zho,zho,528498,Chinese,Chinese,zh,zh_core_web_lg,web,577,"tok2vec, tagger, parser, ner, attribute_ruler"
nan,"nan, zho, eng",522127,Min Nan Chinese,NA,NA,NA,NA,NA,NA


In [71]:
model_grouped_counts %>% 
mutate(spacy_model_exists = ifelse(!is.na(models), TRUE, FALSE)) %>% 
group_by(spacy_model_exists) %>%
summarize(count = sum(utterance_count)) %>%
mutate(prop = count / sum(count))

,spacy_model_exists,count,prop
,<lgl>,<dbl>,<dbl>
1,FALSE,4477921,0.2358474
2,TRUE,14508596,0.7641526


We have 76% coverage for utterances - assuming that the first language listed in the list is the language of the utterance.

In [80]:
model_counts$num_lengths <- lengths(model_counts$all_codes)
sum(model_counts %>% filter(num_lengths >1) %>% select(utterance_count) %>% unlist())

[1] 3986296

In [81]:
3986296/sum(model_counts$utterance_count)

[1] 0.209954